In [ ]:
import cgmquantify as cgm
import pandas as pd
import datetime as datetime
from datetime import date
from scipy.stats import stats
import numpy as np
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from scipy.signal import find_peaks
from scipy.stats import stats

from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from numpy import mean
from numpy import absolute
from numpy import sqrt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!pip install cgmquantify

In [ ]:
!pip install tslearn

In [ ]:
!pip install imbalanced-learn

In [ ]:
df_merged_all = pd.read_csv('df_overnight_and_PRO.csv', sep='\t')


In [ ]:
df_merged_all

In [ ]:
df_merged_all = df_merged_all.dropna(subset=['axis1', 'axis2', 'axis3'],how='all')

df_merged_all['axis1'] = df_merged_all['axis1'].fillna(0)
df_merged_all['axis2'] = df_merged_all['axis2'].fillna(0)
df_merged_all['axis3'] = df_merged_all['axis3'].fillna(0)

df_merged_all['time'] = pd.to_datetime(df_merged_all['time'] ,errors = 'coerce')

df_merged_all = df_merged_all.reset_index(drop=True)
df_merged_all = df_merged_all.drop(["Unnamed: 0", "joinID"], axis=1)

fasting_states_to_keep = ['fasting', 'non-fasting']
df_merged_all = df_merged_all[df_merged_all.label.isin(fasting_states_to_keep)]

#phases_to_keep = ['lTRE', 'eTRE']
#df_merged_all = df_merged_all[df_merged_all.phase.isin(phases_to_keep)]

final_df = df_merged_all.sort_values(by=['id', 'time', "phase"], ascending=True)

In [ ]:
final_df['Time'] = pd.to_datetime(final_df['time'], format='%Y-%m-%dT%H:%M:%S')
final_df['Glucose'] = pd.to_numeric(final_df['gl'])
final_df['Day'] = final_df["Time"].dt.date
final_df = final_df.reset_index(drop=True)

In [ ]:
df_train = final_df[final_df['id'] > 8]
df_test = final_df[final_df['id'] <= 8]

In [ ]:
id_test = [2, 10, 12, 17, 19, 23, 35, 34, 38, 41, 45]

In [ ]:
df_test = final_df.loc[final_df['id'].isin(id_test)] 
df_train = final_df.loc[~final_df['id'].isin(id_test)] 

## Label Encoding

In [ ]:
map_dict = {"non-fasting": 1, "fasting":0}
df_train["label"] = df_train["label"].map(map_dict)
df_test["label"] = df_test["label"].map(map_dict)

In [ ]:
map_dict = {"screening": 1, "lTRE":0, "eTRE":2}
df_train["phase"] = df_train["phase"].map(map_dict)
df_test["phase"] = df_test["phase"].map(map_dict)

In [ ]:
df_train

## Windowing

In [ ]:
x_list_train = []
train_labels = []


window_size = 3
step_size = 1

# creating overlaping windows of size window-size 100
for i in range(0, df_train.shape[0] - window_size, step_size):
    id_p = df_train['id'].values[i: i + window_size]
    phase = df_train['phase'].values[i: i + window_size]
    time = df_train['Time'].values[i: i + window_size]
    times = df_train['Day'].values[i: i + window_size]
    gls = df_train['Glucose'].values[i: i + window_size]
    steps = df_train['steps'].values[i: i + window_size]
    inclineStanding = df_train['inclineStanding'].values[i: i + window_size]
    inclineSitting = df_train['inclineSitting'].values[i: i + window_size]
    inclineLying = df_train['inclineLying'].values[i: i + window_size]
    
    timedelta_first = time[1] - time[0]
    timedelta_second = time[2] - time[1]
    one_hour = 3600000000000
    
    if (timedelta_first < one_hour and timedelta_second < one_hour):
   
    
        xs = df_train['axis1'].values[i: i + window_size]
        ys = df_train['axis2'].values[i: i + window_size]
        zs = df_train['axis3'].values[i: i + window_size]

        label = stats.mode(df_train['label'][i: i + window_size])[0][0]

        df_slice = pd.DataFrame(time, columns=['Time'])
        df_slice["id"] = id_p
        df_slice["phase"] = phase
        df_slice["Day"] = times
        df_slice["Glucose"] = gls   
        df_slice["axis1"] = xs
        df_slice["axis2"] = ys
        df_slice["axis3"] = zs
        df_slice["steps"] = steps
        df_slice["inclineStanding"] = inclineStanding
        df_slice["inclineSitting"] = inclineSitting
        df_slice["inclineLying"] = inclineLying

        x_list_train.append(df_slice)

        train_labels.append(label)

In [ ]:
x_list_test = []
test_labels = []


window_size = 3
step_size = 1

# creating overlaping windows of size window-size 100
for i in range(0, df_test.shape[0] - window_size, step_size):
    id_p = df_test['id'].values[i: i + window_size]
    phase = df_test['phase'].values[i: i + window_size]
    time = df_test['Time'].values[i: i + window_size]
    times = df_test['Day'].values[i: i + window_size]
    gls = df_test['Glucose'].values[i: i + window_size]
    steps = df_train['steps'].values[i: i + window_size]
    inclineStanding = df_test['inclineStanding'].values[i: i + window_size]
    inclineSitting = df_test['inclineSitting'].values[i: i + window_size]
    inclineLying = df_test['inclineLying'].values[i: i + window_size]
    
    timedelta_first = time[1] - time[0]
    timedelta_second = time[2] - time[1]
    one_hour = 3600000000000
    
    if (timedelta_first < one_hour and timedelta_second < one_hour):
    
        xs = df_test['axis1'].values[i: i + window_size]
        ys = df_test['axis2'].values[i: i + window_size]
        zs = df_test['axis3'].values[i: i + window_size]

        label = stats.mode(df_test['label'][i: i + window_size])[0][0]

        df_slice = pd.DataFrame(time, columns=['Time'])
        df_slice["id"] = id_p
        df_slice["phase"] = phase
        df_slice["Day"] = times
        df_slice["Glucose"] = gls     
        df_slice["axis1"] = xs
        df_slice["axis2"] = ys
        df_slice["axis3"] = zs
        df_slice["steps"] = steps
        df_slice["inclineStanding"] = inclineStanding
        df_slice["inclineSitting"] = inclineSitting
        df_slice["inclineLying"] = inclineLying

        x_list_test.append(df_slice)

        test_labels.append(label)

## Feature computation

In [ ]:
def feat_statistical_measures_gl_acc(x_list):
    X_train = pd.DataFrame()

    for df_temp in x_list:
        
        id_p = df_temp.loc[1,'id']
        phase = df_temp.loc[1,'phase']
        time_median = df_temp.loc[1,'Time']
        
        cgm_summary = list(cgm.summary(df_temp))
        cgm_J_index = cgm.J_index(df_temp)

        cgm_interdaycv = cgm.interdaycv(df_temp)

        cgm_summary.append(cgm_J_index)
        cgm_summary.append(cgm_interdaycv)

        
        
        # mean
        z_mean = df_temp["axis3"].mean()

        # max
        y_max = df_temp["axis2"].max()


        # energy
        z_energy = ((df_temp["axis3"] **2) / 100).sum()
        
        general = []
        general.extend([id_p, phase, time_median])


        acc_summary = []
        acc_summary.extend([z_mean, y_max ,z_energy])
        

        features = general
        features.extend(cgm_summary)
        features.extend(acc_summary)

        X_train_temp = pd.DataFrame([features], columns = ["id", "phase","time","mean", "median","minimum", "maximum","first_quartile", "third_quartile",
                                                              "J_index","cgm_interdaycv", 
                                                              "z_mean", "y_max", "z_energy"])

        
        X_train = pd.concat([X_train, X_train_temp], ignore_index=True)
        
    return X_train

In [ ]:
def feat_statistical_measures_gl_tsc(x_list):
    X_train = pd.DataFrame()

    for df_temp in x_list:
        time_median = df_temp.loc[1,'Time']
        cgm_summary = list(cgm.summary(df_temp))
        cgm_LBGI = cgm.LBGI(df_temp)
        cgm_HBGI = cgm.HBGI(df_temp)
        cgm_ADRR = cgm.ADRR(df_temp)
        cgm_GMI = cgm.GMI(df_temp)
        cgm_J_index = cgm.J_index(df_temp)
        cgm_eA1c = cgm.eA1c(df_temp)
        cgm_interdaysd = cgm.interdaysd(df_temp)
        cgm_interdaycv = cgm.interdaycv(df_temp)

        cgm_summary.append(cgm_LBGI)
        cgm_summary.append(cgm_HBGI)
        cgm_summary.append(cgm_ADRR)
        cgm_summary.append(cgm_GMI)
        cgm_summary.append(cgm_J_index)
        cgm_summary.append(cgm_eA1c)
        cgm_summary.append(cgm_interdaysd)
        cgm_summary.append(cgm_interdaycv)




        general = []
        general.extend([time_median])
        general.extend(cgm_summary)

        X_train_temp = pd.DataFrame([general],
                                    columns=["time","mean", "median", "minimum", "maximum", "first_quartile", "third_quartile",
                                             "LBGI", "HBGI", "ADRR", "GMI", "J_index", "eA1c", "interdaysd",
                                             "cgm_interdaycv"])

        X_train = pd.concat([X_train, X_train_temp], ignore_index=True)
        
    return X_train

In [ ]:

def feat_statistical_measures_acc_tsc(x_list):
    X_train = pd.DataFrame()

    for df_temp in x_list:
        time_median = df_temp.loc[1,'Time']

        # mean
        z_mean = df_temp["axis3"].mean()

        # max
        y_max = df_temp["axis2"].max()


        # energy
        z_energy = ((df_temp["axis3"] **2) / 100).sum()

        general = []
        general.extend([time_median])

        acc_summary = []
        acc_summary.extend([z_mean, y_max ,z_energy])
        
        features = general
        features.extend(acc_summary)

        X_train_temp = pd.DataFrame([features],
                                    columns=["time","z_mean", "y_max", "z_energy"])

        X_train = pd.concat([X_train, X_train_temp], ignore_index=True)

    return X_train


In [ ]:

X_test = feat_statistical_measures_gl_acc(x_list_test)

X_train = feat_statistical_measures_gl_acc(x_list_train)

In [ ]:
X_test = feat_statistical_measures_gl_tsc(x_list_test)

X_train = feat_statistical_measures_gl_tsc(x_list_train)

In [ ]:
X_test

In [ ]:
X_test = feat_statistical_measures_acc_tsc(x_list_test)

X_train = feat_statistical_measures_acc_tsc(x_list_train)

## Saving and Loading the featured Data

In [ ]:
y_train = np.array(train_labels)
y_test = np.array(test_labels)

In [ ]:
X_test.to_csv("TSC_data/eval_Xtest.csv", sep='\t')
X_train.to_csv("TSC_data/eval_Xrain.csv", sep='\t')


In [ ]:
y_train_float = y_train
y_test_float = y_test

In [ ]:
np.save('TSC_data/eval_y_train.npy', y_train_float)
np.save('TSC_data/eval_y_test.npy', y_test_float)

In [ ]:
X_test = pd.read_csv('eval_Xtest.csv', sep='\t')
X_train = pd.read_csv('eval_Xrain.csv', sep='\t')

X_test = X_test.drop(["Unnamed: 0"], axis=1)
X_train = X_train.drop(["Unnamed: 0"], axis=1)

In [ ]:
y_train_float = np.load('TSC_data\\eval_y_train.npy')
y_test_float = np.load('TSC_data\\eval_y_test.npy')

In [ ]:
mylist = list(set(y_train_float))
mylist

## Feature Selection

In [ ]:
X_train_rfe = X_train.drop(["time"], axis=1)
X_test_rfe = X_test.drop(["time"], axis=1)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE

# fit random forest model
model = RandomForestRegressor(n_estimators=500, random_state=1)
model.fit(X_train_rfe, y_train_float)
# show importance scores
print(model.feature_importances_)

In [ ]:
from sklearn.feature_selection import RFE

rfe = RFE(RandomForestRegressor(n_estimators=500, random_state=1), n_features_to_select=4)
fit = rfe.fit(X_train_rfe, y_train_float)
# report selected features
print('Selected Features:')


In [ ]:
fit.ranking_

In [ ]:
fit.support_

In [ ]:
for i in range(len(fit.support_)):
    if fit.support_[i]:
        print(names[i])

In [ ]:
names

In [ ]:
names = X_train_rfe.columns.values[0:-1]


In [ ]:
from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
X, y = make_friedman1(n_samples=50, n_features=5, random_state=0)
estimator = SVR(kernel="linear")
selector = RFE(estimator, n_features_to_select=5, step=1)
selector = selector.fit(X_train_rfe, y_train_float)



In [ ]:
selector.ranking_

In [ ]:
n_features_to_select = 10
rfe = RFE(regressor, n_features_to_select=n_features_to_select)
rfe.fit(X_train_rfe, y_train_float)
predictions_rfe = rfe.predict(X_test_rfe)

In [ ]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, predictions_rfe))


In [ ]:
from operator import itemgetter
features = X_train.columns.to_list()
for x, y in (sorted(zip(selector.ranking_ , features), key=itemgetter(0))):
    print(x, y)

## Building the time series

In [ ]:
mylist = list(set(y_test_float))

mylist

### Test-dataset

### acc and gl

In [ ]:
from datetime import datetime


X_test.time = pd.to_datetime(X_test.time)

x_list_test_tsc_1st = []
test_labels_tsc = []
x_list_test_tsc_2nd = []
counter=0


last_time_before = X_test['time'].values[0]

for i in range(0, X_test.shape[0], 1):
    time = X_test['time'].values[i]
    cgm_interdaycv =  X_test['cgm_interdaycv'].values[i]
    J_index = X_test['J_index'].values[i]
    maximum = X_test['maximum'].values[i]
    z_mean = X_test['z_mean'].values[i]
    y_max = X_test['y_max'].values[i]
    z_energy = X_test['z_energy'].values[i]
    
    timedelta_first =  time - last_time_before

    one_hour = 3600000000000
    
    if timedelta_first < one_hour and counter < 5:
        x_list_test_tsc_3rd = []
        x_list_test_tsc_3rd.append(cgm_interdaycv)
        x_list_test_tsc_3rd.append(J_index)
        x_list_test_tsc_3rd.append(maximum)
        x_list_test_tsc_3rd.append(z_mean)
        x_list_test_tsc_3rd.append(y_max)
        x_list_test_tsc_3rd.append(z_energy)
        x_list_test_tsc_2nd.append(x_list_test_tsc_3rd)
    
    else:
        
        if counter == 5:
            x_list_test_tsc_1st.append(x_list_test_tsc_2nd)
            test_labels_tsc.append(y_test_float[i])
            
        counter = 0
        x_list_test_tsc_2nd = [] 
        x_list_test_tsc_3rd = []
        x_list_test_tsc_3rd.append(cgm_interdaycv)
        x_list_test_tsc_3rd.append(J_index)
        x_list_test_tsc_3rd.append(maximum)
        x_list_test_tsc_3rd.append(z_mean)
        x_list_test_tsc_3rd.append(y_max)
        x_list_test_tsc_3rd.append(z_energy)
        x_list_test_tsc_2nd.append(x_list_test_tsc_3rd)
        
    counter += 1
    
    last_time_before = time


### gl

In [ ]:
from datetime import datetime


X_test.time = pd.to_datetime(X_test.time)

x_list_test_tsc_1st = []
test_labels_tsc = []
x_list_test_tsc_2nd = []
counter=0


last_time_before = X_test['time'].values[0]

for i in range(0, X_test.shape[0], 1):
    time = X_test['time'].values[i]
    cgm_interdaycv =  X_test['cgm_interdaycv'].values[i]
    J_index = X_test['J_index'].values[i]
    maximum = X_test['maximum'].values[i]
    
    timedelta_first =  time - last_time_before

    one_hour = 3600000000000
    
    if timedelta_first < one_hour and counter < 5:
        x_list_test_tsc_3rd = []
        x_list_test_tsc_3rd.append(cgm_interdaycv)
        x_list_test_tsc_3rd.append(J_index)
        x_list_test_tsc_3rd.append(maximum)

        x_list_test_tsc_2nd.append(x_list_test_tsc_3rd)
    
    else:
        
        if counter == 5:
            x_list_test_tsc_1st.append(x_list_test_tsc_2nd)
            test_labels_tsc.append(y_test_float[i])
            
        counter = 0
        x_list_test_tsc_2nd = [] 
        x_list_test_tsc_3rd = []
        x_list_test_tsc_3rd.append(cgm_interdaycv)
        x_list_test_tsc_3rd.append(J_index)
        x_list_test_tsc_3rd.append(maximum)
        x_list_test_tsc_2nd.append(x_list_test_tsc_3rd)
        
    counter += 1
    
    last_time_before = time

### acc

In [ ]:
from datetime import datetime


X_test.time = pd.to_datetime(X_test.time)

x_list_test_tsc_1st = []
test_labels_tsc = []
x_list_test_tsc_2nd = []
counter=0


last_time_before = X_test['time'].values[0]

for i in range(0, X_test.shape[0], 1):
    time = X_test['time'].values[i]
    z_mean = X_test['z_mean'].values[i]
    y_max = X_test['y_max'].values[i]
    z_energy = X_test['z_energy'].values[i]
    
    timedelta_first =  time - last_time_before

    one_hour = 3600000000000
    
    if timedelta_first < one_hour and counter < 5:
        x_list_test_tsc_3rd = []

        x_list_test_tsc_3rd.append(z_mean)
        x_list_test_tsc_3rd.append(y_max)
        x_list_test_tsc_3rd.append(z_energy)
        x_list_test_tsc_2nd.append(x_list_test_tsc_3rd)
    
    else:
        
        if counter == 5:
            x_list_test_tsc_1st.append(x_list_test_tsc_2nd)
            test_labels_tsc.append(y_test_float[i])
            
        counter = 0
        x_list_test_tsc_2nd = [] 
        x_list_test_tsc_3rd = []

        x_list_test_tsc_3rd.append(z_mean)
        x_list_test_tsc_3rd.append(y_max)
        x_list_test_tsc_3rd.append(z_energy)
        x_list_test_tsc_2nd.append(x_list_test_tsc_3rd)
        
    counter += 1
    
    last_time_before = time

### Train-Dataset

In [ ]:
from datetime import datetime


X_train.time = pd.to_datetime(X_train.time)


x_list_train_tsc_1st = []
train_labels_tsc = []
x_list_train_tsc_2nd = []
counter = 0


last_time_before = X_train['time'].values[0]



for i in range(0, X_train.shape[0], 1):
    time = X_train['time'].values[i]
    cgm_interdaycv =  X_train['cgm_interdaycv'].values[i]
#     steps =  X_train['steps'].values[i]
#     interdaysd =  X_train['interdaysd'].values[i]
#     cgm_MAGE =  X_train['cgm_MAGE'].values[i]
    J_index = X_train['J_index'].values[i]
    maximum = X_train['maximum'].values[i]
    z_mean = X_train['z_mean'].values[i]
    y_max = X_train['y_max'].values[i]
    z_energy = X_train['z_energy'].values[i]
    
    timedelta_first =  time - last_time_before

    one_hour = 3600000000000
    
    if timedelta_first < one_hour and counter < 5:
        
        x_list_train_tsc_3rd = []
        x_list_train_tsc_3rd.append(cgm_interdaycv)
        x_list_train_tsc_3rd.append(J_index)
        x_list_train_tsc_3rd.append(maximum)
        x_list_train_tsc_3rd.append(z_mean)
        x_list_train_tsc_3rd.append(y_max)
        x_list_train_tsc_3rd.append(z_energy)
        x_list_train_tsc_2nd.append(x_list_train_tsc_3rd)
    
    else:
        if counter == 5:
            x_list_train_tsc_1st.append(x_list_train_tsc_2nd)
            train_labels_tsc.append(y_train_float[i])
            
        counter = 0
        x_list_train_tsc_2nd = [] 
        x_list_train_tsc_3rd = []
        x_list_train_tsc_3rd.append(cgm_interdaycv)
        x_list_train_tsc_3rd.append(J_index)
        x_list_train_tsc_3rd.append(maximum)
        x_list_train_tsc_3rd.append(z_mean)
        x_list_train_tsc_3rd.append(y_max)
        x_list_train_tsc_3rd.append(z_energy)
        x_list_train_tsc_2nd.append(x_list_train_tsc_3rd)
    counter += 1
    last_time_before = time

### gl

In [ ]:
from datetime import datetime


X_train.time = pd.to_datetime(X_train.time)


x_list_train_tsc_1st = []
train_labels_tsc = []
x_list_train_tsc_2nd = []
counter = 0


last_time_before = X_train['time'].values[0]



for i in range(0, X_train.shape[0], 1):
    time = X_train['time'].values[i]
    cgm_interdaycv =  X_train['cgm_interdaycv'].values[i]
    J_index = X_train['J_index'].values[i]
    maximum = X_train['maximum'].values[i]
    
    timedelta_first =  time - last_time_before

    one_hour = 3600000000000
    
    if timedelta_first < one_hour and counter < 5:
        
        x_list_train_tsc_3rd = []
        x_list_train_tsc_3rd.append(cgm_interdaycv)
        x_list_train_tsc_3rd.append(J_index)
        x_list_train_tsc_3rd.append(maximum)

        x_list_train_tsc_2nd.append(x_list_train_tsc_3rd)
    
    else:
        if counter == 5:
            x_list_train_tsc_1st.append(x_list_train_tsc_2nd)
            train_labels_tsc.append(y_train_float[i])
            
        counter = 0
        x_list_train_tsc_2nd = [] 
        x_list_train_tsc_3rd = []
        x_list_train_tsc_3rd.append(cgm_interdaycv)
        x_list_train_tsc_3rd.append(J_index)
        x_list_train_tsc_3rd.append(maximum)

        x_list_train_tsc_2nd.append(x_list_train_tsc_3rd)
    counter += 1
    last_time_before = time

### acc

In [ ]:
from datetime import datetime


X_train.time = pd.to_datetime(X_train.time)


x_list_train_tsc_1st = []
train_labels_tsc = []
x_list_train_tsc_2nd = []
counter = 0


last_time_before = X_train['time'].values[0]



for i in range(0, X_train.shape[0], 1):
    time = X_train['time'].values[i]
    z_mean = X_train['z_mean'].values[i]
    y_max = X_train['y_max'].values[i]
    z_energy = X_train['z_energy'].values[i]
    
    timedelta_first =  time - last_time_before

    one_hour = 3600000000000
    
    if timedelta_first < one_hour and counter < 5:
        
        x_list_train_tsc_3rd = []
        x_list_train_tsc_3rd.append(z_mean)
        x_list_train_tsc_3rd.append(y_max)
        x_list_train_tsc_3rd.append(z_energy)
        x_list_train_tsc_2nd.append(x_list_train_tsc_3rd)
    
    else:
        if counter == 5:
            x_list_train_tsc_1st.append(x_list_train_tsc_2nd)
            train_labels_tsc.append(y_train_float[i])
            
        counter = 0
        x_list_train_tsc_2nd = [] 
        x_list_train_tsc_3rd = []
        x_list_train_tsc_3rd.append(z_mean)
        x_list_train_tsc_3rd.append(y_max)
        x_list_train_tsc_3rd.append(z_energy)
        x_list_train_tsc_2nd.append(x_list_train_tsc_3rd)
    counter += 1
    last_time_before = time

# TS Learn Classification

### To Time Series Dataset

In [ ]:
from tslearn.utils import to_time_series_dataset
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
from tslearn import generators


In [ ]:
X_train =to_time_series_dataset(x_list_train_tsc_1st)
X_test =to_time_series_dataset(x_list_test_tsc_1st)

In [ ]:
y_train =  np.array(train_labels_tsc)
y_test =  np.array(test_labels_tsc)

### Checking for Balance

In [ ]:
from collections import Counter
print(Counter(y_test).keys()) 
print(Counter(y_test).values())
print(Counter(y_train).keys())
print(Counter(y_train).values())

In [ ]:
data2d_test = X_test.reshape(X_test.shape[0], -1)
data2d_train = X_train.reshape(X_train.shape[0], -1)

In [ ]:
print(X_test.shape)
print(X_train.shape)
print(data2d_test.shape)
print(data2d_train.shape)

### Undersampling with Random Selection

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
import collections
rus = RandomUnderSampler(random_state=0, replacement=True)
data2d_test_resampled , y_test_resampled = rus.fit_resample(data2d_test, y_test)
data2d_train_resampled , y_train_resampled = rus.fit_resample(data2d_train, y_train)

In [ ]:
print(data2d_test_resampled.shape)
print(data2d_train_resampled.shape)

In [ ]:
from collections import Counter
print(Counter(y_test_resampled).keys())
print(Counter(y_test_resampled).values()) 
print(Counter(y_train_resampled).keys()) 
print(Counter(y_train_resampled).values())

In [ ]:
data2d_test_resampled.shape[0]

In [ ]:
data3d_X_test = data2d_test_resampled.reshape(-1, X_test.shape[1], X_test.shape[2])
data3d_X_train = data2d_train_resampled.reshape(-1, X_train.shape[1], X_train.shape[2])

In [ ]:
print(data3d_X_test.shape)
print(data3d_X_train.shape)

In [ ]:
def GridSearch_table_plot(grid_clf, param_name,
                          num_results=15,
                          negative=True,
                          graph=True,
                          display_all_params=True):

    '''Display grid search results

    Arguments
    ---------

    grid_clf           the estimator resulting from a grid search
                       for example: grid_clf = GridSearchCV( ...

    param_name         a string with the name of the parameter being tested

    num_results        an integer indicating the number of results to display
                       Default: 15

    negative           boolean: should the sign of the score be reversed?
                       scoring = 'neg_log_loss', for instance
                       Default: True

    graph              boolean: should a graph be produced?
                       non-numeric parameters (True/False, None) don't graph well
                       Default: True

    display_all_params boolean: should we print out all of the parameters, not just the ones searched for?
                       Default: True

    Usage
    -----

    GridSearch_table_plot(grid_clf, "min_samples_leaf")

                          '''
    from matplotlib      import pyplot as plt
    from IPython.display import display
    import pandas as pd

    clf = grid_clf.best_estimator_
    clf_params = grid_clf.best_params_
    if negative:
        clf_score = -grid_clf.best_score_
    else:
        clf_score = grid_clf.best_score_
    clf_stdev = grid_clf.cv_results_['std_test_score'][grid_clf.best_index_]
    cv_results = grid_clf.cv_results_

    print("best parameters: {}".format(clf_params))
    print("best score:      {:0.5f} (+/-{:0.5f})".format(clf_score, clf_stdev))
    if display_all_params:
        import pprint
        pprint.pprint(clf.get_params())

    # pick out the best results
    # =========================
    scores_df = pd.DataFrame(cv_results).sort_values(by='rank_test_score')

    best_row = scores_df.iloc[0, :]
    if negative:
        best_mean = -best_row['mean_test_score']
    else:
        best_mean = best_row['mean_test_score']
    best_stdev = best_row['std_test_score']
    best_param = best_row['param_' + param_name]

    # display the top 'num_results' results
    # =====================================
    display(pd.DataFrame(cv_results) \
            .sort_values(by='rank_test_score').head(num_results))

    # plot the results
    # ================
    scores_df = scores_df.sort_values(by='param_' + param_name)

    if negative:
        means = -scores_df['mean_test_score']
    else:
        means = scores_df['mean_test_score']
    stds = scores_df['std_test_score']
    params = scores_df['param_' + param_name]

    # plot
    if graph:
        plt.figure(figsize=(8, 8))
        plt.errorbar(params, means, yerr=stds)

        plt.axhline(y=best_mean + best_stdev, color='red')
        plt.axhline(y=best_mean - best_stdev, color='red')
        plt.plot(best_param, best_mean, 'or')

        plt.title(param_name + " vs Score\nBest Score {:0.5f}".format(clf_score))
        plt.xlabel(param_name)
        plt.ylabel('Score')
        plt.show()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

## KNeighborsTimeSeriesClassifier

In [ ]:
import pickle
start = "path_models_TSC\\"
end = ".pkl"
folder = "gl\\"
ml_type = "KNN_TSC"

path = start + folder + ml_type + end

with open(path, "rb") as f:
    model = pickle.load(f)

In [ ]:
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
knn = KNeighborsTimeSeriesClassifier(n_neighbors=2)
knn.fit(data3d_X_train, y_train_resampled)

In [ ]:
# set search parameters
n_neighbors = [int(x) for x in np.linspace(start = 1, stop = 100, num = 50)]   
weights = ['uniform','distance']
metric = ['euclidean','manhattan','chebyshev','minkowski'] 
random_grid = {
    'n_neighbors': n_neighbors,
    'weights': weights,
    'metric': metric,
}

In [ ]:
knn_random = RandomizedSearchCV(estimator = knn, random_state = 42,n_jobs = -1,param_distributions = random_grid,n_iter = 20, cv=3,verbose = 2)
knn_random.fit(data3d_X_train, y_train_resampled)
knn_random.best_params_

In [ ]:
knn_random.best_params_

best_estimator = knn_random.best_estimator_

best_estimator.fit(data3d_X_train, y_train_resampled)

pred = best_estimator.predict(data3d_X_test)

accuracy_score(y_test_resampled, pred )

In [ ]:
pred = model.predict(data3d_X_test)

accuracy_score(pred, y_test_resampled)

In [ ]:
accuracy_score(y_test_resampled, smoothing(pred))

In [ ]:
print("\n -------------Classification Report-------------\n")
print(classification_report(y_test_resampled, pred))

In [ ]:
GridSearch_table_plot(knn_random, "weights", negative=False)

In [ ]:
y_pred = knn.predict(data3d_X_test)
accuracy_score(y_test_resampled, y_pred)

In [ ]:
from sklearn import metrics

confusion_matrix = metrics.confusion_matrix(y_test_resampled, smoothing(y_pred))

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()


In [ ]:
print("\n -------------Classification Report-------------\n")
print(classification_report(y_test_resampled, smoothing(y_pred)))

In [ ]:
import pickle
with open('TSC_data/models_TSC/acc/KNN_TSC.pkl','wb') as f:
    pickle.dump(knn,f)

## TimeSeriesSVC

In [ ]:
from tslearn.svm import TimeSeriesSVC
clf = TimeSeriesSVC(C=1.0, kernel="gak", gamma=0.001)
clf.fit(data3d_X_train,y_train_resampled)

In [ ]:
y_pred = clf.predict(data3d_X_test)
accuracy_score(y_test_resampled, y_pred)

In [ ]:
parameters = {'kernel': ['rbf', 'sigmoid'],
             'C': [1.0, 100.0, 1000.0],
             'gamma': [0.001, 0.01,0.1]}#


In [ ]:
random_estimator = RandomizedSearchCV(estimator = clf,
                                   param_distributions = parameters,
                                   cv = 5,
                                   n_iter = 10,
                                   scoring = 'accuracy',
                                   n_jobs = -1,
                                   verbose = 1, 
                                   random_state = 1,
                                  )

random_estimator.fit(data3d_X_train, y_train_resampled)

In [ ]:
random_estimator.best_params_

best_estimator = random_estimator.best_estimator_

best_estimator.fit(data3d_X_train, y_train_resampled)

pred = best_estimator.predict(data3d_X_test)

accuracy_score(pred, y_test_resampled)

In [ ]:
GridSearch_table_plot(random_estimator, "gamma", negative=False)

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test_resampled, pred))

In [ ]:
from sklearn import metrics

confusion_matrix = metrics.confusion_matrix(y_test_resampled, smoothing(y_pred))

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()

In [ ]:
import pickle
with open('TSC_data/models_TSC/both/CLF_TSC.pkl','wb') as f:
    pickle.dump(clf,f)

## Clustering TimeSeriesKMeans

In [ ]:
from tslearn.clustering import TimeSeriesKMeans
km = TimeSeriesKMeans(n_clusters=2, metric="dtw")
labels = km.fit_predict(data3d_X_train)


km_bis = TimeSeriesKMeans(n_clusters=2, metric="softdtw")
labels_bis = km_bis.fit_predict(data3d_X_train)

In [ ]:
pred_df = pd.DataFrame({"actual":y_train_resampled, "prediction":labels})